# 🎯 GitHub Issues Tracker & Resolver

This notebook helps you:
1. **Track open GitHub issues** and their sub-issues
2. **Display issues in a table** for easy selection
3. **Pick specific issues** to resolve using our agent system
4. **Monitor resolution progress** in real-time

Created: August 17, 2025

In [15]:
# GitHub Issues Tracker and Resolver
# ====================================
# This notebook helps track and resolve GitHub issues using our agent system

import requests
import pandas as pd
import json
import os
from datetime import datetime
import sys
from pathlib import Path

# Add helpers to path
helpers_path = Path.cwd() / "helpers"
sys.path.insert(0, str(helpers_path))

print("🎯 GitHub Issues Tracker and Resolver")
print("=" * 50)

# GitHub repository configuration
REPO_OWNER = "Uh-X3L"
REPO_NAME = "kip-retl-uh-x3l"
GITHUB_API_BASE = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}"

# Check if we have GitHub token (optional for public repos)
github_token = os.environ.get('GITHUB_TOKEN')
headers = {'Authorization': f'token {github_token}'} if github_token else {}

print(f"📋 Repository: {REPO_OWNER}/{REPO_NAME}")
print(f"🔑 GitHub Token: {'✅ Available' if github_token else '❌ Not set (using public access)'}")

🎯 GitHub Issues Tracker and Resolver
📋 Repository: Uh-X3L/kip-retl-uh-x3l
🔑 GitHub Token: ❌ Not set (using public access)


In [16]:
def fetch_github_issues():
    """Fetch all open issues from the GitHub repository."""
    try:
        # Fetch open issues
        url = f"{GITHUB_API_BASE}/issues"
        params = {
            'state': 'open',
            'per_page': 100,
            'sort': 'created',
            'direction': 'desc'
        }
        
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        
        issues = response.json()
        print(f"✅ Found {len(issues)} open issues")
        
        return issues
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching issues: {e}")
        return []

def parse_issue_relationships(issues):
    """Parse issues and identify main issues vs sub-issues."""
    main_issues = []
    sub_issues = []
    
    for issue in issues:
        title = issue['title']
        body = issue.get('body', '')
        number = issue['number']
        
        # Check if this is a sub-issue (mentioned in other issues)
        is_sub_issue = False
        parent_issue = None
        
        # Look for references to this issue in other issues
        for other_issue in issues:
            other_body = other_issue.get('body', '')
            other_title = other_issue.get('title', '')
            
            # Check if this issue is referenced as a sub-issue
            if (f"#{number}" in other_body or 
                f"issue #{number}" in other_body.lower() or
                f"sub-issue #{number}" in other_body.lower()):
                is_sub_issue = True
                parent_issue = other_issue['number']
                break
        
        # Categorize the issue
        issue_data = {
            'number': number,
            'title': title,
            'state': issue['state'],
            'created_at': issue['created_at'],
            'updated_at': issue['updated_at'],
            'url': issue['html_url'],
            'labels': [label['name'] for label in issue.get('labels', [])],
            'assignee': issue['assignee']['login'] if issue.get('assignee') else None,
            'body_preview': body[:100] + '...' if len(body) > 100 else body
        }
        
        if is_sub_issue:
            issue_data['parent_issue'] = parent_issue
            sub_issues.append(issue_data)
        else:
            # Look for sub-issues mentioned in this issue's body
            referenced_issues = []
            for other_issue in issues:
                other_number = other_issue['number']
                if (f"#{other_number}" in body and other_number != number):
                    referenced_issues.append(other_number)
            
            issue_data['sub_issues'] = referenced_issues
            main_issues.append(issue_data)
    
    return main_issues, sub_issues

# Fetch and parse issues
print("🔍 Fetching GitHub issues...")
all_issues = fetch_github_issues()

if all_issues:
    main_issues, sub_issues = parse_issue_relationships(all_issues)
    print(f"📋 Main Issues: {len(main_issues)}")
    print(f"📝 Sub-Issues: {len(sub_issues)}")
else:
    main_issues, sub_issues = [], []

🔍 Fetching GitHub issues...
✅ Found 48 open issues
📋 Main Issues: 40
📝 Sub-Issues: 8


In [18]:
def select_issue_to_resolve(issue_number):
    """Select a specific issue to resolve using our agent system."""
    
    # Find the selected issue
    selected_issue = None
    for issue in all_issues_data:
        if issue['number'] == issue_number:
            selected_issue = issue
            break
    
    if not selected_issue:
        print(f"❌ Issue #{issue_number} not found!")
        return None
    
    print(f"\n🎯 SELECTED ISSUE #{issue_number}")
    print("=" * 50)
    print(f"📋 Title: {selected_issue['title']}")
    print(f"📅 Created: {selected_issue['created_at']}")
    print(f"👤 Assignee: {selected_issue['assignee']['login'] if selected_issue.get('assignee') else 'Unassigned'}")
    print(f"🏷️  Labels: {', '.join([label['name'] for label in selected_issue.get('labels', [])])}")
    print(f"🔗 URL: {selected_issue['html_url']}")
    
    # Show issue body (description)
    body = selected_issue.get('body', '')
    if body:
        print(f"\n📝 Description:")
        print("-" * 20)
        print(body[:500] + "..." if len(body) > 500 else body)
    
    # Find related sub-issues
    sub_issues_for_this = []
    if 'sub_issues_table' in globals() and not sub_issues_table.empty:
        # Find sub-issues that reference this issue
        for _, row in sub_issues_table.iterrows():
            if row['Parent Issue #'] == issue_number:
                sub_issues_for_this.append(row)
    
    if sub_issues_for_this:
        print(f"\n📝 Related Sub-Issues ({len(sub_issues_for_this)}):")
        print("-" * 30)
        for sub_issue in sub_issues_for_this:
            print(f"   ↳ #{sub_issue['Issue #']}: {sub_issue['Title']}")
    
    return selected_issue

def resolve_issue_with_agents(issue_number):
    """Use our agent system to resolve the selected issue."""
    
    # Import our agent system
    try:
        from helpers.backend_supervisor_role_tools import BackendSupervisorAgent
        from helpers.simple_messaging import create_simple_messaging
        from helpers.simple_agent_coordinator import SimpleAgentCoordinator
        
        print(f"\n🤖 RESOLVING ISSUE #{issue_number} WITH AGENT SYSTEM")
        print("=" * 60)
        
        # Get the selected issue details
        selected_issue = select_issue_to_resolve(issue_number)
        if not selected_issue:
            return False
        
        print(f"\n🚀 Starting agent-based resolution...")
        print(f"   📋 Issue: {selected_issue['title']}")
        print(f"   🎯 Strategy: Using Backend Supervisor + Specialized Agents")
        print(f"   📡 Communication: Redis messaging")
        
        # Here you would integrate with your existing agent system
        print(f"\n⚡ NEXT STEPS:")
        print(f"   1. Initialize Backend Supervisor Agent")
        print(f"   2. Create specific task plan for issue #{issue_number}")
        print(f"   3. Deploy specialized agents via Redis")
        print(f"   4. Execute coordinated resolution")
        print(f"   5. Validate and update GitHub issue")
        
        return True
        
    except ImportError as e:
        print(f"❌ Error importing agent system: {e}")
        print(f"   Make sure the helpers modules are available")
        return False

# Interactive selection function
print("\n🔧 ISSUE RESOLUTION INTERFACE")
print("=" * 50)
print("Use the following functions to interact with issues:")
print()
print("📋 To select an issue:")
print("   selected_issue = select_issue_to_resolve(ISSUE_NUMBER)")
print("   Example: select_issue_to_resolve(114)")
print()
print("🤖 To resolve with agents:")
print("   resolve_issue_with_agents(ISSUE_NUMBER)")
print("   Example: resolve_issue_with_agents(114)")
print()
print("📊 Available data:")
print(f"   main_issues_table: {len(main_issues_table) if 'main_issues_table' in globals() and main_issues_table is not None else 0} main issues")
print(f"   sub_issues_table: {len(sub_issues_table) if 'sub_issues_table' in globals() and sub_issues_table is not None else 0} sub-issues")
print(f"   all_issues_data: {len(all_issues_data) if 'all_issues_data' in globals() else 0} total issues")


🔧 ISSUE RESOLUTION INTERFACE
Use the following functions to interact with issues:

📋 To select an issue:
   selected_issue = select_issue_to_resolve(ISSUE_NUMBER)
   Example: select_issue_to_resolve(114)

🤖 To resolve with agents:
   resolve_issue_with_agents(ISSUE_NUMBER)
   Example: resolve_issue_with_agents(114)

📊 Available data:
   main_issues_table: 40 main issues
   sub_issues_table: 8 sub-issues
   all_issues_data: 48 total issues


In [20]:
# Display Issues in Organized Format
# ===================================
# Show main issues with their sub-issues grouped together

def display_organized_issues():
    """Display issues organized by main issue with sub-issues grouped underneath."""
    
    if 'main_issues_table' not in globals() or main_issues_table is None:
        print("❌ No issues data available. Run the previous cells first.")
        return
    
    print("\n📋 ORGANIZED ISSUES VIEW")
    print("=" * 70)
    print(f"Total: {len(main_issues_table)} main issues, {len(sub_issues_table) if 'sub_issues_table' in globals() and sub_issues_table is not None else 0} sub-issues")
    print()
    
    # Display each main issue with its sub-issues
    for idx, main_issue in main_issues_table.iterrows():
        issue_num = main_issue['Issue #']
        
        print(f"🔸 ISSUE #{issue_num}: {main_issue['Title']}")
        print(f"   📅 Created: {main_issue['Created']}")
        print(f"   🏷️  Labels: {', '.join(main_issue['Labels']) if main_issue['Labels'] else 'None'}")
        print(f"   👤 Assignee: {main_issue['Assignee'] or 'Unassigned'}")
        print(f"   🔗 {main_issue['URL']}")
        
        # Find and display sub-issues for this main issue
        if 'sub_issues_table' in globals() and sub_issues_table is not None and not sub_issues_table.empty:
            related_subs = sub_issues_table[sub_issues_table['Parent Issue #'] == issue_num]
            
            if not related_subs.empty:
                print(f"   📝 Sub-Issues ({len(related_subs)}):")
                for _, sub_issue in related_subs.iterrows():
                    print(f"      ↳ #{sub_issue['Issue #']}: {sub_issue['Title']}")
                    print(f"        📅 {sub_issue['Created']} | 🏷️ {', '.join(sub_issue['Labels']) if sub_issue['Labels'] else 'None'}")
                    print(f"        🔗 {sub_issue['URL']}")
            else:
                print(f"   📝 Sub-Issues: None")
        else:
            print(f"   📝 Sub-Issues: None")
        
        print()  # Empty line between issues
    
    print("=" * 70)
    print("💡 To work with a specific issue, use:")
    print("   select_issue_to_resolve(ISSUE_NUMBER)")
    print("   resolve_issue_with_agents(ISSUE_NUMBER)")

# Display the organized view
display_organized_issues()


📋 ORGANIZED ISSUES VIEW
Total: 40 main issues, 8 sub-issues

🔸 ISSUE #117: 🧪 Basic Validation
   📅 Created: 2025-08-17
   🏷️  Labels: subtask, agent-testing, complexity-medium, backend-supervisor-agent
   👤 Assignee: Unassigned
   🔗 https://github.com/Uh-X3L/kip-retl-uh-x3l/issues/117
   📝 Sub-Issues (1):
      ↳ #114: 🎯 Comprehensive Codebase Review and Fix Implementation - Full Implementation Plan
        📅 2025-08-17 | 🏷️ ai-project, has-subtasks, needs-worker, needs-testing, complexity-medium, backend-supervisor-agent
        🔗 https://github.com/Uh-X3L/kip-retl-uh-x3l/issues/114

🔸 ISSUE #116: 🔨 Core Feature Implementation
   📅 Created: 2025-08-17
   🏷️  Labels: subtask, agent-worker, complexity-medium, backend-supervisor-agent
   👤 Assignee: Unassigned
   🔗 https://github.com/Uh-X3L/kip-retl-uh-x3l/issues/116
   📝 Sub-Issues: None

🔸 ISSUE #115: 🔨 Minimal Setup
   📅 Created: 2025-08-17
   🏷️  Labels: subtask, agent-worker, complexity-medium, backend-supervisor-agent
   👤 Assigne

In [21]:
# In a new cell:
resolve_issue_with_agents(114)  # This will use your agent coordination system

🔨 Worker Agent module loaded successfully!
🧪 Testing Agent module loaded successfully!
📚 Documentation Agent module loaded successfully!

🤖 RESOLVING ISSUE #114 WITH AGENT SYSTEM

🎯 SELECTED ISSUE #114
📋 Title: 🎯 Comprehensive Codebase Review and Fix Implementation - Full Implementation Plan
📅 Created: 2025-08-17T11:50:42Z
👤 Assignee: Uh-X3L
🏷️  Labels: ai-project, has-subtasks, needs-worker, needs-testing, complexity-medium, backend-supervisor-agent
🔗 URL: https://github.com/Uh-X3L/kip-retl-uh-x3l/issues/114

📝 Description:
--------------------
# Comprehensive Codebase Review and Fix Implementation

## 📋 Project Overview

Research topic: Comprehensive Codebase Review and Fix Implementation. Azure AI web research not available.

**Estimated Complexity:** Medium  
**Total Estimated Hours:** 10.0h  
**Number of Subtasks:** 3
**Agent Types Required:** testing, worker

## 🎯 Requirements


    OVERVIEW:
    Systematically review and fix all identified issues in the codebase to make it produ

True